##### Name:Bakri Badet
##### Matrikelnr:1125983

In [34]:
import pandas as pd
import numpy as np
import requests
import json

In [35]:
start_date="2017-10-04"
base="https://api-web.nhle.com/v1/schedule/"
end_date="2018-04-08"

In [36]:
# Function to retrieve the data:
def retrieve_data(date):
    r=requests.get(url=base+date)
    return r.json()

In [37]:
# Ids all my games in season 2017/2018
ids=[]
date=start_date
while True:
    data=retrieve_data(date)
    for week in range(len(data["gameWeek"])):
        for game in range(len(data["gameWeek"][week]["games"])):
            if (data["gameWeek"][week]["games"][game]["season"]) == 20172018 :
                ids.append(data["gameWeek"][week]["games"][game]["id"])
    next_date=data["nextStartDate"]
    if next_date>end_date:
        break
    else:
        date=next_date    
    

In [64]:
len(ids)


1274

In [39]:
box_base="https://api-web.nhle.com/v1/gamecenter/"
end_point="/boxscore"
def get_game_record(id):
    r=requests.get(
        url=box_base+str(id)+end_point
    )
    return r.json()

In [66]:
x=get_game_record(2017020001)
x.keys()


dict_keys(['id', 'season', 'gameType', 'limitedScoring', 'gameDate', 'venue', 'venueLocation', 'startTimeUTC', 'easternUTCOffset', 'venueUTCOffset', 'tvBroadcasts', 'gameState', 'gameScheduleState', 'periodDescriptor', 'regPeriods', 'awayTeam', 'homeTeam', 'clock', 'playerByGameStats', 'summary', 'gameOutcome'])

In [42]:
GamesData=[]
for id in ids:
    data= get_game_record(id)
    GamesData.append(data)
       

In [46]:
len(GamesData)

1274

In [78]:
data=pd.json_normalize(GamesData,record_path=["summary",'teamGameStats'],meta=["id","season","gameDate",'gameState'])
data

,category,awayValue,homeValue,id,season,gameDate,gameState
0,sog,31,37,2017020001,20172018,2017-10-04,OFF
1,faceoffWinningPctg,0.555556,0.444444,2017020001,20172018,2017-10-04,OFF
2,powerPlay,2/4,0/8,2017020001,20172018,2017-10-04,OFF
3,powerPlayPctg,0.5,0.0,2017020001,20172018,2017-10-04,OFF
4,pim,16,8,2017020001,20172018,2017-10-04,OFF
...,...,...,...,...,...,...,...
11461,pim,8,6,2017020616,20172018,2018-04-08,OFF
11462,hits,15,20,2017020616,20172018,2018-04-08,OFF
11463,blockedShots,20,7,2017020616,20172018,2018-04-08,OFF
11464,giveaways,9,11,2017020616,20172018,2018-04-08,OFF


In [77]:
data_pivot=data.pivot_table(index=["id","season","gameDate","gameState"],
                            columns="category",values=["homeValue","awayValue"],aggfunc="first").reset_index()
data_pivot

id    season    gameDate gameState    awayValue  \
category                                             blockedShots   
0         2017020001  20172018  2017-10-04       OFF           24   
1         2017020002  20172018  2017-10-04       OFF           19   
2         2017020003  20172018  2017-10-04       OFF           17   
3         2017020004  20172018  2017-10-04       OFF           17   
4         2017020005  20172018  2017-10-05       OFF           10   
...              ...       ...         ...       ...          ...   
1269      2017021270  20172018  2018-04-07       OFF           15   
1270      2017021271  20172018  2018-04-07       OFF           22   
1271      2017040631  20172018  2018-01-28     FINAL            0   
1272      2017040632  20172018  2018-01-28     FINAL            1   
1273      2017040633  20172018  2018-01-28     FINAL            3   

                                                          ...            \
category faceoffWinningPctg giveaways hits pim powerPlay  ... takeaways   
0                  0.555556        10   16  16       2/4  ...         1   
1                  0.411765         5   36  10       1/4  ...         3   
2                  0.460317        18   29   9       0/1  ...        11   
3                  0.517241         9   21  12       3/5  ...         5   
4                  0.432432        12   25  15       2/7  ...        10   
...                     ...       ...  ...  ..       ...  ...       ...   
1269               0.584906         6   11   6       0/1  ...         2   
1270               0.481481        11   15   4       0/1  ...         9   
1271               0.333333         0    0   0       0/0  ...         6   
1272               0.583333         2    0   0       0/0  ...         2   
1273               0.526316         1    0   1       0/0  ...         7   

            homeValue                                                  \
category blockedShots faceoffWinningPctg giveaways hits pim powerPlay   
0                  14           0.444444         7   18   8       0/8   
1                  10           0.588235        13   27   8       1/5   
2                  18           0.539683        16   31   7       0/2   
3                  17           0.482759        20   14  10       2/6   
4                   7           0.567568         7   23  19       1/5   
...               ...                ...       ...  ...  ..       ...   
1269                6           0.415094        11   15   2       0/3   
1270               23           0.518519         4   16   2       0/2   
1271                0           0.666667         0    0   0       0/0   
1272                0           0.416667         4    1   0       0/0   
1273                2           0.473684         3    0   0       0/1   

                                      
category powerPlayPctg sog takeaways  
0                  0.0  37         5  
1                  0.2  33         4  
2                  0.0  45        18  
3             0.333333  35        11  
4                  0.2  32        13  
...                ...  ..       ...  
1269               0.0  36         2  
1270               0.0  30         5  
1271               0.0  17         6  
1272               0.0  14         9  
1273               0.0  11         6  

[1274 rows x 22 columns]

In [17]:
#data["gameWeek"][0]["games"][i]["id"]

2017020001

In [14]:
# from datetime import datetime, timedelta

# start_date = datetime.strptime("2017-10-04", "%Y-%m-%d")
# end_date = datetime.strptime("2018-04-08", "%Y-%m-%d")
# lst=[]
# current_date = start_date

# while current_date <= end_date:
#     lst.append(current_date.strftime("%Y-%m-%d"))
#     current_date += timedelta(days=7)

In [1]:
print("1123"<"1124")

True
